In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

###### Importing Necessary libraries

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
%matplotlib inline
warnings.filterwarnings('ignore')


In [ ]:
data=pd.read_csv('../input/insurance/insurance.csv')

#### Let's just print the First 5 rows to get an overview of how the Data Looks

In [ ]:
data.head()

##### ***isnull*** function is used to check if there is any null values in Dataset and counting null values if there is any.

In [ ]:
data.isnull().sum()

Check the type of data in the Dataset field,we can see that the Dataset contains both numerical and categorical values.

In [ ]:
data.info()

Since Dataset is of Numerical and Categorical Data, Let's convert categorical values into Numerical for easy computing and Extract Information on the Dataset. Before that Let's find out what are the Unique values in categorical Columns

In [ ]:
categorical=['sex','smoker','region']

for col in categorical:
    print(col,'Column has {} as unique values'.format(data[col].unique()))

In [ ]:
from sklearn.preprocessing import LabelEncoder
Le=LabelEncoder()

for col in categorical:
    Le.fit(data[col].drop_duplicates())
    data[col]=Le.transform(data[col])
data.head()

The Dataset is now converted into Numerical Data

In [ ]:
data.info()

Let's Find the Descriptive statistics of the Dataset

In [ ]:
data.describe()

In [ ]:
data.hist(figsize=(15,12))


by histogram we can get to know the Distribution of the data for each column

In [ ]:
data.corr()

The Correlation function corr() computes pairwaise correlation of each column,The values ranges from -1 to +1 where -1 indicates it is higly negativel correlated which has zero effect on the outcome whereas +1 indicates the data is higly possitively correlated which means, the outcome is higly dependent on that Variable. 


Correlation function plotted with heatmap

In [ ]:
plt.figure(figsize=(10,7))
sns.heatmap(data.corr(),annot=True,cmap='Oranges')

By the heatmpa figure we can see that charges and smoker are higly correlated which means that the people who smokes end up in hospital paying high charges for treatment.

whereas the age and bmi body mass index is slightly correlated by which we can believe that, the charges varies depending on the age of the people

In [ ]:
sns.pairplot(data)

In [ ]:
#plot distribution of charges for smokers
f=plt.figure(figsize=(15,7))
ax=f.add_subplot(121)
sns.distplot(data[data.smoker==1]['charges'],color='c',ax=ax)
ax.set_title('Distribution of charges for smokers')

#plot distribution of charges for non smokers
ax=f.add_subplot(122)
sns.distplot(data[data.smoker==0]['charges'],color='b')
ax.set_title('Distribution of charges for non smokers')

The charges for smokers is quite high as compared with non smokers.

In [ ]:
sns.countplot(x='smoker',hue='sex',data=data)

# sex:          
male=1                       

female=0                      

# smoker:
 smoker=1
 
 non-smoker=0
 
count of male smokers is quite high as compared with female smokers

In [ ]:
sns.catplot(x='sex',y='charges',hue='smoker',kind='violin',data=data,palette='magma')

In [ ]:
plt.figure(figsize=(15,5))
sns.boxplot(x='charges',y='smoker',data=data[data.sex==1],orient='h')

In [ ]:
plt.figure(figsize=(15,5))
sns.boxplot(x='charges',y='smoker',data=data[data.sex==0],orient='h',palette='rainbow')

The Median Charges for Male is Higher as compared with Female Smokers.

In [ ]:
plt.figure(figsize=(10,7))
plt.title('Dist of age')
sns.distplot(data['age'])

In [ ]:
#Count of smokers who are exactly 18 years old 
sns.countplot(x='smoker',hue='sex',data=data[data['age']==18])

In [ ]:
sns.lmplot(x='age',y='charges',hue='smoker',data=data,size=8)

In [ ]:
#Count of People who has Children and Smokes
sns.countplot(x='smoker',hue='sex',data=data[data.children>0])

The count of people who has Children and Smoke are less as compared with people who smokes.

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import PolynomialFeatures
from sklearn.metrics import r2_score,mean_squared_error,roc_auc_score
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import StandardScaler

In [ ]:
#Train the Model with Linear Regression and check the accuracy

X=data.drop(['charges'],axis=1)
y=data.charges

X_train,X_test,y_train,y_test=train_test_split(X,y,random_state=0)

# std=StandardScaler()
# X_train=std.fit_transform(X_train)
# X_test=std.transform(X_test)


lr=LinearRegression()
lr.fit(X_train,y_train)
test=lr.predict(X_test)
r2_score(y_test,test)

In [ ]:
from sklearn.linear_model import Ridge,Lasso

for alpha in [0.0001,0.001,0.01,0.1,1,10,100]:
    Rid=Ridge(alpha=alpha).fit(X_train,y_train)
    rid_test=Rid.predict(X_test)
    print('Accuracy scorer for alpha {} is '.format(alpha),r2_score(y_test,rid_test))

In [ ]:
#Training the model with Lasso with different alpha values
for alpha in [0.0001,0.001,0.01,0.1,1,10,100]:
    Las=Lasso(alpha=alpha).fit(X_train,y_train)
    las_test=Las.predict(X_test)
    print('Accuracy scorer for alpha {} is '.format(alpha),r2_score(y_test,las_test))

Best accuracy is 79 when compared with Liner models, Let's check with Different Models

We can apply increase the number of features by applying polynomial features with the required degree, Which generates a new feature matrix of polynomial combination.

In [ ]:
from sklearn.preprocessing import PolynomialFeatures

poly=PolynomialFeatures(2)

poly_train=poly.fit_transform(X_train)
poly_test=poly.transform(X_test)


In [ ]:
from sklearn.tree import DecisionTreeRegressor

tree=DecisionTreeRegressor()
tree.fit(poly_train,y_train)
train_predict=tree.predict(poly_train)
print('train_score',r2_score(y_train,train_predict))
tree_test_result=tree.predict(poly_test)
print('test_score',r2_score(y_test,tree_test_result))


The model is Overfitting, We can apply gridsearchCV method to find the best parameters for the model

In [ ]:
from sklearn.model_selection import GridSearchCV

parameters=[{'min_samples_split':[2,5,15,25,50,60,70,80,100],'criterion':['mse','friedman_mse','mae']}]

grid=GridSearchCV(estimator=tree,param_grid=parameters,cv=10,n_jobs=-1)

grid=grid.fit(X_train,y_train)

In [ ]:
grid.best_score_

In [ ]:
grid.best_estimator_

In [ ]:
from sklearn.tree import DecisionTreeRegressor

tree=DecisionTreeRegressor(min_samples_split=30,criterion='mae')
tree.fit(poly_train,y_train)
tree_test_result=tree.predict(poly_test)
print('test_score',r2_score(y_test,tree_test_result)*100)


In [ ]:
from sklearn.model_selection import cross_validate

score=cross_validate(tree,X,y,cv=10,scoring='r2',return_train_score=True)

In [ ]:
score['test_score'].mean()

In [ ]:
from sklearn.tree import DecisionTreeRegressor

spl=[]
sco=[]
for split in [2,5,10,20,30,40,50,100,150,200,300]:
    tree=DecisionTreeRegressor(min_samples_split=split,criterion='mae')
    tree.fit(poly_train,y_train)
    tree_test_result=tree.predict(poly_test)
    test_score=r2_score(y_test,tree_test_result)
    spl.append(split)
    sco.append(test_score)


In [ ]:
plt.figure(figsize=(12,8))
plt.plot(spl,sco)
plt.title('Test score compared with min_samples_split')
plt.xlabel('min_samples_split')
plt.ylabel('Test Score')

From the figure we can see that as the min_samples_split is increased the test score is increased after 40 min_samples_split the tests score result starts decreasing